In [24]:
import tensorflow as tf
'''
tensorflow中的基本数据类型是张量（标量（0） 向量（1） 矩阵（2），各种维度的向量（d>2））
张量最重要的三个属性是形状 类型 和 值
tensorflow的数据类型
tf.int32
tf.int64
tf.float32
tf.float64
bool
string
'''
tf.__version__ # 查看tensorflow版本
tf.__path__ # 查看tensorflow路径

['/home/lxy/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow']

In [17]:
# 创建张量
exp = tf.random.uniform(shape=(2,2),dtype=tf.float64)
exp1 = tf.fill([2,2],9)

In [16]:
exp.shape
exp.dtype
exp.numpy()
exp

array([[0.45947365, 0.48582582],
       [0.75320882, 0.65855576]])

In [19]:
# tensor和numpy数据类型的转换
exp_numpy = exp.numpy()
exp_tensor = tf.convert_to_tensor(exp_numpy)

In [22]:
# 取值操作
i = tf.range(10)
i = tf.random.shuffle(i)

<tf.Tensor: id=38, shape=(2, 2), dtype=int32, numpy=
array([[9, 9],
       [9, 9]], dtype=int32)>

In [26]:
# 梯度下降初探 -- numpy 
import numpy as np
X_raw = np.array([2013,2014,2015,2016,2017],dtype=np.float32)
y_raw = np.array([12000,14000,15000,16500,17500],dtype=np.float32)
X = (X_raw -X_raw.min()) / (X_raw.max() - X_raw.min())
y = (y_raw - y_raw.min()) / (y_raw.max() - y_raw.min())

a,b = 0,0
num_epoch =1000
learning_rate = 5e-4
# 梯度下降 求使得误差最小时的w和b这两个参数
# 目标函数是 (y_pred - y)**2 分别对 a 和 b求导
for e in range(num_epoch):
    y_pred = a*X+b
    grad_a,grad_b = 2 * (y_pred - y).dot(X), 2*(y_pred-y).sum()
    a,b = a - learning_rate * grad_a, b - learning_rate * grad_b
print(a,b)

0.535206765532493 0.30109002557396913


In [28]:
# 梯度下降初探-- Tensorflow
X = tf.constant(X)
y = tf.constant(y)
a = tf.Variable(initial_value=0.)
b = tf.Variable(initial_value=0.)
variables = [a,b]
num_epoch = 100000
optimizer = tf.keras.optimizers.SGD(learning_rate=5e-4)
for e in range(num_epoch):
    with tf.GradientTape() as tape:
        y_pred = a * X + b
        loss = tf.reduce_sum(tf.square(y_pred-y))
    grads = tape.gradient(loss,variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads,variables))

print(a,b)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.98176914> <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.054570343>


In [ ]:
'''
在 tensorflow 中 推荐使用tf.keras 构建模型 
其中最重要的两个概念是 模型 Model 和 层 Layer
'''
# 模型初探
import tensorflow.keras.Model as Model

class MyModel(Model):
    def __init__(self):
        super().__init__()
    
    def call(self,input):
        output = 0
        return output

In [ ]:
'''
tf.keras.layers.Dense(
    inputs=64,  # 输入该网络层的数据
    units=10,  # 输出的维度大小
    activation=None,  # 选择使用的（激活函数）
    use_bias=True,  # 是否使用（偏置项）
    kernel_initializer=None,  # 卷积核的初始化器
    bias_initializer=tf.zeros_initializer(),  # 偏置项的初始化器
    kernel_regularizer=None,  # 卷积核的正则化
    activaty_regularizer=None,  # 偏置项的正则化
    kernel_constraint=None,  # 对主权重矩阵进行约束
    bias_constraint=None,  # 对偏置向量进行约束
    trainable=True,  # 可以设置为不可训练，（冻结）网络层
    name=None,  # 层的名字
    reuse=None  # 是否重复使用参数
)

激活函数是 tf.nn.
'''
class Linear(Model):
    def __init__(self):
        super().__init__()
        self.dense = tf.keras.layers.Dense(
            units=1,
            activation=None, # 激活函数
            use_bias=True,
            kernel_initializer=tf.zeros_initializer(),
            bias_initializer=tf.zeros_initializer()    
        )
        
    def call(self,input):
        output = self.dense(input)
        return output

X = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
y = tf.constant([[10.0], [20.0]])
model = Linear()
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-2) # 优化器


for i in range(100):
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.reduce_mean(tf.square(y_pred-y))
    grads = tape.gradient(loss,model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads,model.variables))
print(model.variables)
      

In [ ]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         # [batch_size, 28, 28, 1]
        x = self.flatten(inputs)    # [batch_size, 784]
        x = self.dense1(x)          # [batch_size, 100]
        x = self.dense2(x)          # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output
        
lambda